In [1]:
import os
import gym
import math
import torch
import numpy as np
from typing import List
from tqdm.auto import tqdm
from datetime import datetime
from collections import defaultdict
from torch.distributions import Categorical
from torch.utils.tensorboard import SummaryWriter
from nltk.translate.gleu_score import sentence_gleu

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
%cd ..
import src.envs
from src.search import search_best_actions
from src.models.seq2labels import PretrainedEncoder, Seq2Labels
from src.utils import remove_ansi, iterative_prediction, load_json, write_json, discount_cumsum, freeze_params, scale, is_gce_instance
%cd notebooks

/home/rajk/Machine_Learning/DRL-GEC
/home/rajk/Machine_Learning/DRL-GEC/notebooks


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
@torch.no_grad()
def select_action(policy, state, reference, mask_generator, explore=False):
    [logits] = policy([state])
    dist = Categorical(logits=logits)
    action_np = search_best_actions(policy, state, reference, mask_generator, explore=explore)
    action = torch.from_numpy(action_np).to(logits.device)
    log_pi = dist.log_prob(action)
    return action_np, log_pi, dist.entropy()

def get_log_pi(policy, state, action):
    [logits] = policy([state])
    dist = Categorical(logits=logits)
    action = torch.from_numpy(action).to(logits.device)
    log_pi = dist.log_prob(action)
    return log_pi

In [6]:
def get_evaluator(data_path, label_vocab, num_iterations=10):
    json_data = load_json(data_path)
    src_ref = ((data_dict["text"], data_dict["references"]) for data_dict in json_data) 
    sources, references = zip(*src_ref)
    print(f"Number of evaluation examples: {len(sources)}")
    del json_data
    
    def eval_func(policy):
        policy.eval()
        predictions = iterative_prediction(policy, label_vocab, sources, num_iter=num_iterations, insert_start=True, verbose=False)
        score = np.mean([sentence_gleu(refs, pred) for refs, pred in zip(references, predictions)])
        policy.train()
        return score
    
    return eval_func

In [7]:
def train(pbar, optim, grad_scaler, policy, buffer, batch_size=32, focal_alpha=0.25, focal_gamma=2.0):
    num_items = len(buffer["return"])
    accumulation_size = math.ceil(num_items/batch_size)
    # Set up the progress bar
    pbar.reset()
    pbar.total = num_items
    # buffer["return"] = scale(buffer["return"])
    buffer["return"] = torch.tensor(buffer["return"], device=device)
    losses = []
    optim.zero_grad()
    for i in range(0, num_items, batch_size):
        state_batch = buffer["state"][i:i+batch_size]
        action_batch = buffer["action"][i:i+batch_size]
        return_batch = buffer["return"][i:i+batch_size]
        focal_log_pis = []
        for state, action in zip(state_batch, action_batch):                    # Obtain log_pi(state, action) for the batch
            log_pi = get_log_pi(policy, state, action)
            focal_coef = focal_alpha*(1-log_pi.exp()).pow(focal_gamma)
            focal_log_pi = (focal_coef*log_pi).sum()                      # Sum log_probs over tokens               
            focal_log_pis.append(focal_log_pi)
        focal_log_pis = torch.stack(focal_log_pis)
        pi_loss = -(focal_log_pis*return_batch).mean()
        grad_scaler.scale(pi_loss/accumulation_size).backward()
        losses.append(pi_loss.item())
        pbar.update(len(state_batch))
    grad_scaler.step(optim)
    grad_scaler.update()
    pbar.refresh()
    return np.mean(losses)

# Define parameters

In [8]:
# Algorithm parameters
eps = 0.5
min_eps = 0.25
eps_decay = 1.0 # 0.999995
gamma = 0.9
focal_alpha = 0.25
focal_gamma = 2.0
# Training parameters
cold_lr = 1e-3
warm_lr = 1e-5
lr = cold_lr
batch_size = 32
update_interval = 500
num_unfreeze_layers = 0            # Unfreeze only the last 98 layers
dropout = 0.1
weight_decay = 0.0
episodes = 100_000
cold_episodes = 0 # 20_000
# Evaluation parameters
eval_max_iter = 10
evaluate_interval = 5000
record_output_interval = 10
dev_data_path = r"../data/processed/wi+locness/dev.json"
model_path = "sl_logs/pretrain_synthetic_31:10:2022_19:25/model-best.pt"
train_type = "pretrain" if model_path is None else "finetune"
current_datetime = datetime.now().strftime("%d_%m_%Y_%H:%M")
log_dir = os.path.join("pg_logs_new", f"{train_type}_rl_{current_datetime}")
env_kwargs = {
    "id": "gec_lev_dist-v1",
    "datasets": ["wi+locness"],                  # Datasets to load
    "correct_examples_percent": [1.0],           # Percentage of correct sentences to load
    "repeat": 1,                                 # Number of repetation of each sentence in ``
    "repeat_interval": 1000,                     # Interval to use for repetition
    "consecutive": False,                        # If the repetition should have consecutive or random distribution
}
meta_data = {
    "base_model": model_path,
    "env_config": env_kwargs,
    "description": """
    Finetune the FL (2M) pretrained model. 
    Dropout enabled.
    positive reward for success; episodes ends when all keep predicted for correct sentence
    Don't shuffle batches.
    Explore-Exploit determined per episode.
    Label = UKNONWN if len(candidate_labels) == 0
    Expoit = sample from candidate_probs.
    Explore = Sample from candidate_lev_probs + [$KEEP].
    No rescaling and Negative rewards. +1.0 for success.
    Early termination for false negative too.
    eval_max_iter set from 5 to 10.
    Check all labels for candidate labels for replace edits
    Reward Scale = 1.0.
    """
}

# Load environment

In [9]:
env = gym.make(new_step_api=True, **env_kwargs)

Original number of data in wi+locness: 26815
Number of data without correct sentences: 26815


# Load the models

In [10]:
model_name = "roberta-base"
tokenizer_config = {"use_fast": True}
transformer_config = {
    "output_attentions": False,
    # "hidden_dropout_prob": 0.0,
    # "attention_probs_dropout_prob": 0.0,
}
encoder = PretrainedEncoder(model_name, tokenizer_config, transformer_config).to(device)
policy = Seq2Labels(encoder_model=encoder, num_labels=env.action_space.n, dropout=dropout).to(device)
if model_path:
    policy.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(policy.parameters(), lr=lr, weight_decay=weight_decay)
grad_scaler = torch.cuda.amp.GradScaler()
writer = SummaryWriter(log_dir=log_dir)
evaluator = get_evaluator(dev_data_path, env.labels, eval_max_iter)
write_json(os.path.join(log_dir, "meta.json"), meta_data)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of evaluation examples: 3590


# Train the model

In [ ]:
policy.train()
# Freeze encoder weights
freeze_params(policy.encoder, requires_grad=False, optim=optim, lr=lr)
# Log hyperparameters
writer.add_scalar("hyperparameters/gamma", gamma, 0)
writer.add_scalar("hyperparameters/dropout", dropout, 0)
writer.add_scalar("hyperparameters/batch_size", batch_size, 0)
writer.add_scalar("hyperparameters/focal_alpha", focal_alpha, 0)
writer.add_scalar("hyperparameters/focal_gamma", focal_gamma, 0)
writer.add_scalar("hyperparameters/cold_episodes", cold_episodes, 0)
writer.add_scalar("hyperparameters/update_interval", update_interval, 0)
# Variables for training progress bars
policy_pbar = None

mask_generator = env.mask_generator
max_eval_score = 0
eval_score = evaluator(policy)
writer.add_scalar("rl/eval_score", eval_score, 1)
buffer_dict = defaultdict(list)
for episode in tqdm(range(1, episodes+1), desc="Training Episodes", total=episodes):
    if ((cold_episodes == 0) and (episode == 1)) or (episode == cold_episodes):                # When cold epoch ends, update learning rate and unfreeze certain encoder layers 
        lr = warm_lr
        freeze_params(policy.encoder, requires_grad=True, num_layers=num_unfreeze_layers, optim=optim, lr=lr)
    rewards = []
    log_pis = []
    entropies = []
    token_lens = []
    done = False
    init_state = state = env.reset()
    reference = env.current_reference
    explore = np.random.uniform() < eps
    with torch.cuda.amp.autocast():
        while not done:
            action, log_pi, entropy = select_action(policy, state, reference, mask_generator, explore=explore)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            # Save timestep data
            rewards.append(reward)
            log_pis.append(log_pi)
            entropies.append(entropy)
            token_lens.append(len(next_state))
            buffer_dict["state"].append(state)
            buffer_dict["action"].append(action)
            state = next_state
    # Compute returns
    returns = discount_cumsum(rewards, discount=gamma)
    buffer_dict["return"].extend(returns)
    # eps = max(eps*eps_decay, min_eps)
    # Train the model
    if (episode % update_interval) == 0:
        if policy_pbar is None:
            policy_pbar = tqdm(desc="Updating Policy")
        loss = train(policy_pbar, optim, grad_scaler, policy, buffer_dict, batch_size=batch_size, focal_alpha=focal_alpha, focal_gamma=focal_gamma)
        writer.add_scalar("rl/mean_loss", loss, episode)
        buffer_dict = defaultdict(list)
        torch.cuda.empty_cache()
    # Log the episode output to the tensorboard
    if (episode % record_output_interval) == 0:
        render_output = "  \n".join(remove_ansi(out) for out in env.render())
        writer.add_text("rl/output", render_output, episode)
    # Evaluate the model
    if (episode % evaluate_interval) == 0:
        eval_score = evaluator(policy)
        if eval_score >= max_eval_score:
            torch.save(policy.state_dict(), os.path.join(log_dir, "model-best.pt"))
            max_eval_score = eval_score
        writer.add_scalar("rl/eval_score", eval_score, episode)
    # Log scalar episode results
    rewards = np.array(rewards)
    writer.add_scalar("rl/lr", lr, episode)
    writer.add_scalar("rl/eps", eps, episode)
    writer.add_scalar("rl/explore", explore, episode)
    writer.add_scalar("rl/episode_length", len(rewards), episode)
    writer.add_scalar("rl/episode_reward_last", rewards[-1], episode)
    writer.add_scalar("rl/episode_reward_total", sum(rewards), episode)
    writer.add_scalar("rl/token_length_delta_ratio", (len(state)-len(init_state))/len(init_state), episode)
    # Log histogram episode results
    writer.add_histogram("rl/episode_reward", rewards, episode)
    writer.add_histogram("rl/episode_returns", returns, episode)
    writer.add_histogram("rl/episode_log_pi", torch.cat(log_pis), episode)
    writer.add_histogram("rl/episode_entropy", torch.cat(entropies), episode)
    writer.add_histogram("rl/episode_token_length", np.array(token_lens), episode)
    writer.add_histogram("rl/episode_returns_normalized", np.array(scale(returns)), episode)

Number of frozen parameters: 197/197


Training Episodes:   0%|          | 0/100000 [00:00<?, ?it/s]

Number of frozen parameters: 0/197


/home/rajk/miniconda3/envs/drl-gec/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:133: UserWarning: WARN: The obs returned by the `reset()` method should be an int or np.int64, actual type: <class 'list'>
  logger.warn(f"{pre} should be an int or np.int64, actual type: {type(obs)}")
/home/rajk/miniconda3/envs/drl-gec/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/rajk/miniconda3/envs/drl-gec/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/home/rajk/miniconda3/envs/drl-gec/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:133: UserWarning: WARN: The obs returned by the `s

Updating Policy: 0it [00:00, ?it/s]

# Save model

In [ ]:
torch.save(policy.state_dict(), os.path.join(log_dir, "model-last.pt"))

policy.load_state_dict(torch.load(os.path.join(log_dir, "model-best.pt")))

# Test model

In [ ]:
@torch.no_grad()
def greedy_action(policy, state):
    [logits] = policy([state])
    v, i = logits.topk(5)
    v = v.cpu().numpy()
    i = i.cpu().numpy()
    dist = Categorical(logits=logits)
    probs = dist.probs
    entropy = dist.entropy().cpu().numpy()
    for a, e, lp in zip(state, entropy, zip(env.labels[i], v)):
        print(f"{e:4f}, {a:15}", " -- ".join(f"{l} [{p:5.2f}]" for (l, p) in zip(*lp)))
    print()
    action = logits.argmax(axis=-1)
    # action = Categorical(logits=logits).sample()
    return action.cpu().numpy()

In [ ]:
_ = policy.eval()          # Set poliy to eval mode to disable dropouts

In [ ]:
env = gym.make("gec_lev_dist-v1", datasets=["wi+locness"], new_step_api=True, correct_examples_percent=[0.0], repeat=1)

In [ ]:
state = env.reset()
print("# References")
for ref in env.reference_tokens_list:
    print(ref)
print()
done = False
while not done:
    action = greedy_action(policy, state)
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    state = next_state
    outputs = env.render()
    for o in outputs:
        print(o)

# Close Google Compute Instance

In [ ]:
if is_gce_instance():
    !gcloud compute instances stop drl-gec --zone us-west1-b